In [1]:
import numpy as np

import tensorflow as tf

!pip install -q tensorflow-hub
!pip install -q tfds-nightly

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("version:", tf.__version__)
print("Eager mode", tf.executing_eagerly())
print("Hub version:", hub.__version__)

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
version: 2.1.0
Eager mode True
Hub version: 0.8.0


In [2]:
train_data, validation_data, test_data = tfds.load(
name="imdb_reviews",
split=('train[:60%]', 'train[:40%]','test'),
as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXJOU06/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXJOU06/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteXJOU06/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_data

<PrefetchDataset shapes: ((), ()), types: (tf.string, tf.int64)>

In [4]:
t_e_batch, t_e_label = next(iter(train_data.batch(10)))

In [5]:
t_e_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [6]:
t_e_label

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(t_e_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [10]:
history = model.fit(train_data.shuffle(10000).batch(512),
                   epochs=20,
                   validation_data=validation_data.batch(512),
                   verbose=1)

Epoch 1/20
30/30 [==============================] - 9s 301ms/step - loss: 0.9386 - accuracy: 0.5350 - val_loss: 0.7175 - val_accuracy: 0.5698
Epoch 2/20
30/30 [==============================] - 8s 261ms/step - loss: 0.6741 - accuracy: 0.5807 - val_loss: 0.6512 - val_accuracy: 0.5947
Epoch 3/20
30/30 [==============================] - 7s 239ms/step - loss: 0.6344 - accuracy: 0.6171 - val_loss: 0.6199 - val_accuracy: 0.6213
Epoch 4/20
30/30 [==============================] - 6s 211ms/step - loss: 0.6015 - accuracy: 0.6493 - val_loss: 0.5841 - val_accuracy: 0.6676
Epoch 5/20
30/30 [==============================] - 7s 246ms/step - loss: 0.5649 - accuracy: 0.6845 - val_loss: 0.5475 - val_accuracy: 0.6988
Epoch 6/20
30/30 [==============================] - 7s 230ms/step - loss: 0.5293 - accuracy: 0.7180 - val_loss: 0.5088 - val_accuracy: 0.7380
Epoch 7/20
30/30 [==============================] - 7s 243ms/step - loss: 0.4878 - accuracy: 0.7495 - val_loss: 0.4708 - val_accuracy: 0.7651
Epoch 

In [11]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name,value))

loss: 0.317
accuracy: 0.860


In [15]:
test_s, test_label = next(iter(train_data.batch(1)))
print(test_s)

tf.Tensor([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."], shape=(1,), dtype=string)


In [16]:
result_single = model.predict(test_s)

In [17]:
print(result_single)

[[-7.235136]]


In [18]:
print(test_label)

tf.Tensor([0], shape=(1,), dtype=int64)


In [33]:
test_data_10, test_label_10 = next(iter(train_data.batch(10)))
result_multiple = model.predict(test_data_10)

for i, x in enumerate(result_multiple):
    print("predicted %s %.3f %.3f " % (i, x, test_label_10[i]))

predicted 0 -7.235 0.000
predicted 1 -4.917 0.000
predicted 2 1.743 0.000
predicted 3 2.998 1.000
predicted 4 0.800 1.000
predicted 5 5.182 1.000
predicted 6 -3.255 0.000
predicted 7 -1.270 0.000
predicted 8 -2.729 0.000
predicted 9 -2.560 0.000


In [35]:
test_data_10[2]

<tf.Tensor: shape=(), dtype=string, numpy=b'Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Mountie telling the people of Dawson City, Yukon to elect themselves a marshal (yes a marshal!) and to enforce the law themselves, then gunfighters battling it out on the streets for control of the town? <br /><br />Nothing even remotely resembling that happened on the Canadian side of the border during the Klondike gold rush. Mr. Mann and company appear to have mistaken Dawson City for Deadwood, the Canadian North for the American Wild West.<br /><br />Canadian viewers be prepared for a Reefer Madness type of enjoyable howl with this ludicrous plot, or, to shake your head in disgust.'>